**为什么 FSDP 比 DDP 好**

DDP 是 每个GPU 上放一份完整的模型副本，前向和反向时都在本地算，最后用 all-reduce 来同步梯度。通信少，自然训练速度高，但更容易 OOM。以 https://huggingface.co/blog/zh/pytorch-fsdp 中提到的，在使用 GPT-2 XL (1.5B) 时，即使 batch size 为 1，DDP 也会失败并出现 OOM 错误（2 张 24GB 英伟达 Titan RTX GPU）；而 FSDP 可以支持以更大的 batch size 训练 GPT-2 Large 模型，还可以使用较大的 batch size 训练 DDP 训练不了的 GPT-2 XL 模型。

FSDP (FullyShardedDataParallel) 之所以叫 Fully，就是因为它把模型参数、梯度、优化器状态都 shard（切分） 到不同 GPU 上，每张卡只保存自己负责的一部分；前向需要时再 all-gather 拼回参数；反向后再 reduce-scatter 梯度，最后释放显存。显存占用大大降低（近似 1/N，如果有 N 张卡）。HuggingFace 的 Accelerate 和 DeepSpeed（类似 Zero Stage 3）都在用。但因为有通信，训练速度可能不如 DDP，且代码配置更复杂，对 checkpointing、activation checkpoint 配合要求高。



In [40]:
import torch
import torch.nn as nn
import torch.optim as optim
from accelerate import Accelerator
from torch.utils.data import DataLoader


accelerator = Accelerator()
model = nn.Linear(20, 2).to(accelerator.device)
optimizer = optim.Adam(model.parameters(), lr=0.001, decoupled_weight_decay=True, weight_decay=1e-2)
dataloader = DataLoader(torch.randn(1000, 20).to(accelerator.device), batch_size=100, shuffle=True)

In [41]:
model, optimizer, dataloader = accelerator.prepare(model, optimizer, dataloader)
optimizer

AcceleratedOptimizer (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    decoupled_weight_decay: True
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0.01
)

In [42]:
import time
start_time = time.time()

for i, batch in enumerate(dataloader):
    optimizer.zero_grad()
    outputs = model(batch)
    loss = 0.1 * outputs.sum()
    print(f"step {i} loss: {loss.item()}")
    accelerator.backward(loss)
    optimizer.step()

end_time = time.time()
print(f"Training time: {end_time - start_time:.4f} seconds")

step 0 loss: 0.21575644612312317
step 1 loss: -1.9314888715744019
step 2 loss: 1.2854886054992676
step 3 loss: -1.0365784168243408
step 4 loss: -1.563402533531189
step 5 loss: -0.46200060844421387
step 6 loss: -0.8780922293663025
step 7 loss: -1.7737677097320557
step 8 loss: -0.6850461363792419
step 9 loss: 1.2737587690353394
Training time: 0.0332 seconds


In [43]:
model_vanilla = nn.Linear(20, 2).to("cuda")
optimizer_vanilla = optim.Adam(model_vanilla.parameters(), lr=0.001, weight_decay=1e-2)
dataloader_vanilla = DataLoader(torch.randn(1000, 20).to("cuda"), batch_size=100, shuffle=True)

import time
start_time = time.time()

for i, batch in enumerate(dataloader_vanilla):
    optimizer_vanilla.zero_grad()
    outputs = model_vanilla(batch)
    loss = 0.1 * outputs.sum()
    print(f"step {i} loss: {loss.item()}")
    loss.backward()
    optimizer_vanilla.step()

end_time = time.time()
print(f"Training time: {end_time - start_time:.4f} seconds")

step 0 loss: 3.0945417881011963
step 1 loss: 2.743745803833008
step 2 loss: 2.1409804821014404
step 3 loss: 1.7734102010726929
step 4 loss: 1.342269778251648
step 5 loss: 1.5121549367904663
step 6 loss: 2.2957770824432373
step 7 loss: 3.3115932941436768
step 8 loss: 2.401805877685547
step 9 loss: 1.9694957733154297
Training time: 0.0092 seconds


具体而言，以 GPT-2 为例。假设我们有一个 2 层的 GPT-2（极简化）
- 层 1：Embedding (30k vocab × 768 dim ≈ 23M 参数)
- 层 2：Transformer Block (Attention + MLP ≈ 40M 参数)
- 总参数 ≈ 63M

如果我们用 4 张 GPU 来训练。

**DDP**
- 每张卡 完整的 63M 参数；
- 每张卡算自己 batch 的 loss & backward；
- 梯度通过 all-reduce 聚合，保证 4 张卡的参数保持一致；
- 显存开销：参数 + 梯度 + 优化器状态（比如 Adam 需要额外存 m, v 两个同大小张量）；
- 所以单卡显存大约是 3 × 63M；

**显存大小的实际计算**
- 注意，4 倍参数大小是理论最大开销；
    - 在 PyTorch 里，梯度通常是和参数共享一块内存空间（或至少是相同大小的一份 buffer）；
    - 优化器更新时是 in-place，因此不会额外再复制一份 grad；
    - param (1x) + grad (1x 但复用 buffer) + m (1x) + v (1x) = 3 倍左右；

- PyTorch 里 param 和 grad 的存储关系
    - param：模型权重，比如存成 torch.float32 或 torch.bfloat16；
    - grad：是 param 的 .grad，它的 dtype 和 param 一样；即如果参数是 bf16，反向计算出的梯度也是 bf16；
    - 内存上，PyTorch 的 autograd 通常会分配一块 buffer，大小等于 param 的大小，用来存梯度；
    - 这个 buffer 可以是独立开辟的，也可能和 param 共用 allocator 的 memory pool，但不会直接 alias 到 param 的存储空间（否则 param 会被覆盖）；
    - 复用指的是显存管理策略（memory pool），而不是字节级别共用；
    - param.dtype == grad.dtype，这是 PyTorch 的硬性约定，不会出现 param=bf16 而 grad=int32 这种情况；

- 如果有混合精度 (fp32 / bf16 / fp16 / 量化)
    - 参数和梯度保持同 dtype
        - param 用 bf16，grad 就是 bf16；
        - AdamW 更新时，需要把梯度 cast 到 FP32 master param 上；如果直接用 bf16 梯度去更新，数值范围太小，很容易 underflow / overflow；
    - 参数低精度 + master 参数高精度
        - param 存 bf16 / fp16（节省显存，forward / backward 加速）；
        - 同时维护一份 FP32 的 master param（做优化器更新）；
        - grad 会先 cast 到 FP32 再做更新；
        - NVIDIA Apex、DeepSpeed ZeRO、FSDP mixed precision 都是这样做的；
    - 量化（int8 / int4）模型
        - 推理时，param 可能以 int8 存储，但计算时需要 dequantize 回到 fp16 / bf16 / fp32；
        - 训练时几乎不会直接把 param 存 int8 的；
    - 优化器状态（m, v）一定是 FP32
        - 回顾一下 Adam 公式
        $$\begin{array}{l}m_{t}=\beta_{1} m_{t-1}+\left(1-\beta_{1}\right) g_{t} \\v_{t}=\beta_{2} v_{t-1}+\left(1-\beta_{2}\right) g_{t}^{2} \\\theta_{t+1}=\theta_{t}-\eta \frac{m_{t}}{\sqrt{v_{t}}+\epsilon}\end{array}$$

In [47]:
import torch

# param in bf16
param = torch.randn(1000, dtype=torch.bfloat16, device="cuda", requires_grad=True)

# grad is bf16 too
loss = param.sum()
loss.backward()
print(param.grad.dtype)  # torch.bfloat16

# keep one FP32 master param
master_param = param.detach().clone().float()  

# cast grad to FP32
grad_fp32 = param.grad.float()
print(grad_fp32.dtype)

# update master param with FP32 grad
master_param -= 1e-3 * grad_fp32  

# write back to model's bf16 param
param.data.copy_(master_param.to(torch.bfloat16))
param.data.dtype, param.grad.dtype

torch.bfloat16
torch.float32


(torch.bfloat16, torch.bfloat16)

**FSDP**

FSDP 会把 参数 + 梯度 + 优化器状态 全部分片（shard），不同 GPU 只保留一份子集。
- 参数存储
    - Embedding (23M 参数)，如果 4 张卡：
        - GPU0: 0 – 5.75M
        - GPU1: 5.75M – 11.5M
        - GPU2: 11.5M – 17.25M
        - GPU3: 17.25M – 23M
    - Transformer Block (40M 参数)，同理，每卡 10M；
- 前向计算
    - 当进入 Transformer Block 前，FSDP 会 all-gather 需要的参数（把 40M 的块临时拼回完整）；
    - 也就是说，每张 GPU 拿到所有 shard 的副本，在 all-gather 结束后，每张卡都拥有完整的那一层权重；
    - 因为每张卡都要独立完成前向和反向计算（各自算自己 batch 的 loss），数据都不一样；
    - 为什么显存不会爆，这里的显存峰值是怎么控制住的？
        - 按层 all-gather → 只在用到这一层时，才把 shard 聚合成完整参数；
        - 用完马上释放（forward / backward 一结束就 free 掉）；
- 反向计算
    - 每张卡算自己的梯度，但只保留属于自己 shard 的那部分；
    - FSDP 会做 reduce-scatter：把不同卡上的梯度分配到对应 shard，最后每张卡只保存自己负责的梯度；
- 优化器状态
    - Adam 优化器需要两个状态：m（动量）和 v（二阶矩估计）；
    - 在 FSDP 下，m, v 也被切片，每张卡只保存 15.75M × 2；

FSDP 显存节省 ≈ 4 倍。